In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

# import path variables
from src.config import data_path_interim, data_path_processed
# import simulation parameters
from src.config import simulation_parameters

#
from src.parameters.areal_surface_parameter import getRoughnessParams
from src.parameters.persistence_parameters import getPersistence, persistence_parameters
from src.postprocessing import wrapper_postprocess

In [13]:
# load data, get the according values and delete the loaded data again
fn = f'surface_numSimulation_{simulation_parameters["calculation_method"]}.npz'
data = np.load(data_path_interim / fn)
height = data['values']
x_grid = data['x_grid']
y_grid = data['y_grid']

fn = f'surface_numSimulation_newcoverage_{simulation_parameters["calculation_method"]}.csv'
df_info = pd.read_csv(data_path_interim / fn)

Due to the setup of the FE modelling, we have not introduced any tilting in the surface sample.

Potsprocessing therefore does not need an nominal surface to be subtracted from the surface.

Only step to be done is to apply Gaussian filtering with an cut-off of 0.08mm using FastFourier transform (with zero padding).

In [14]:
processed = simulation_parameters['simulation_processed']
print('Processing of the surface is done using:', processed)

# since we already interpolated the surface we do not do that again
# during postprocessing
interpolation_size = simulation_parameters['interpolation_size_postprocess']

use_filter = False
removeplane = False
if 'filt' in processed:
    use_filter = True
if 'nom' in processed:
    removeplane = True
    
print('Remove a least square fitted plane from surface:', removeplane)
print('Use gausian filter:', use_filter)
print('Inteprolation postproces:', interpolation_size)

Processing of the surface is done using: filt
Remove a least square fitted plane from surface: False
Use gausian filter: True
Inteprolation postproces: 0


In [15]:
# postprocessing; use gaussian filter and do a least square fit
# do not interpolate the heightmap again

t0 = time.time()
znew, xnew, ynew = wrapper_postprocess(height,
                                       x_grid,
                                       y_grid,
                                       removeplane=removeplane,
                                       use_filter=use_filter,
                                       interpolation_size=interpolation_size,
                                       cut_matrix=2,
                                       dimension=2,
                                       cut_off=simulation_parameters['cut_off_wavelength'])
print(f'Postprocessing took {time.time() - t0} seconds.')

Postprocessing took 65.86734080314636 seconds.


In [16]:
# Now calculate the parameter values for the conventional parameters
t0 =time.time()

df_conventional = getRoughnessParams(znew, xnew, ynew,
                                     version='all',
                                     shift=True)
df_conventional['id'] = range(znew.shape[0])

print(f'Calculating conventional parameters took {time.time() - t0} seconds.')

Calculating conventional parameters took 39.865631103515625 seconds.


In [17]:
#now calculate the persistence parameters
t0 =time.time()

#TODO add parameter to say what we do with the infinite class
# right now: set infinite death time to maximum value of heightmap
pers = getPersistence(znew, reduced_homology=False, coeff=2)

print(f'Calculating persistence took {time.time() - t0} seconds.')
t0 =time.time()

names, params = persistence_parameters(pers)
df_persistence = pd.DataFrame(data=params, columns=names)
df_persistence['id'] = range(znew.shape[0])

print(f'Calculating persistence parameters took {time.time() - t0} seconds.')

Calculating persistence took 123.40217804908752 seconds.
Calculating persistence parameters took 11.895437717437744 seconds.


In [18]:
# now merge all data and assign a processed value
df_merged = df_info.merge(df_conventional,
                          on=['id'], how='outer')
df_merged = df_merged.merge(df_persistence,
                            on=['id'], how='outer')
df_merged['processed'] = processed

In [19]:
# now merge all data and assign a processed value
df_merged = df_info.merge(df_conventional,
                          on=['id'], how='outer')
df_merged = df_merged.merge(df_persistence,
                            on=['id'], how='outer')
df_merged['processed'] = processed

In [20]:
# save data
fn = f'numSimulation_parameters_{simulation_parameters["calculation_method"]}.csv'
df_merged.to_csv(data_path_processed / fn,
                index=False)

fn = f'diagrams_{simulation_parameters["calculation_method"]}.npy'
np.save(data_path_processed / fn,
        pers, allow_pickle=False)